<a href="https://colab.research.google.com/github/Sudoyan118/llmatch_kadai/blob/main/LLMATCH_kadai_Gemma3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## はじめに

本ノートブックでは、Googleが提供するマルチモーダルな指示チューニング済みモデル google/gemma-3-4b-it を対象に、以下の2つのタスクに取り組むことを目的とした。


**タスク1:事前学習済みモデルの評価**

google/gemma-3-4b-it を以下の日本語VLM評価用ベンチマークにおいて評価する：
- Heron-Bench
- JA-VLM-Bench-In-the-Wild
- JA-VG-VQA-500

**タスク2:ファインチューニングと再評価**

日本語ビジュアル質問応答データセット line-corporation/JIC-VQA を用いて google/gemma-3-4b-it をファインチューニングし、上記3つのベンチマークを用いて再度評価を行う。</br></br>
しかし、最終的には全てのタスクを完了することはできなかった。
現時点で実施できた内容は以下の通りとなっている：
* google/gemma-3-4b-itをJA-VLM-Bench-In-the-WildとJA-VG-VQA-500のベンチマークを使って、Rouge-Lで評価する


## ライブラリのインストールとバージョン確認

In [ ]:
!pip install -q transformers accelerate datasets peft bitsandbytes gcsfs evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
  

In [ ]:
# バージョン確認したいライブラリ名
target_modules = [
    "transformers",
    "torch",
    "datasets",
    "accelerate",
    "peft",
    "bitsandbytes",
    "fsspec",
    "gcsfs",
    "evaluate",
    "rouge_score"
]

print("=== ライブラリバージョン一覧 ===")

for module_name in target_modules:
    try:
        mod = __import__(module_name)
        print(f"{module_name}: {mod.__version__}")
    except ImportError:
        print(f"{module_name}: インストールされていません")
    except AttributeError:
        print(f"{module_name}: `__version__` が存在しません")



=== ライブラリバージョン一覧 ===
transformers: 4.50.3
torch: 2.6.0+cu124
datasets: 3.5.0
accelerate: 1.5.2
peft: 0.14.0
bitsandbytes: 0.45.5
fsspec: 2024.12.0
gcsfs: 2024.12.0
evaluate: 0.4.3
rouge_score: `__version__` が存在しません


version属性で取得できないライブラリに対して個別でpip showコマンドを用いて確認する

In [ ]:
!pip show rouge_score

Name: rouge_score
Version: 0.1.2
Summary: Pure python implementation of ROUGE-1.5.5.
Home-page: https://github.com/google-research/google-research/tree/master/rouge
Author: Google LLC
Author-email: rouge-opensource@google.com
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, nltk, numpy, six
Required-by: 


## Hugging Faceのアクセストークンログイン

In [ ]:
from huggingface_hub import login
from google.colab import userdata
login(userdata.get("HF_token"))

## モデル準備



In [ ]:
# ---------- ライブラリのインポート ----------
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from datasets import load_dataset
import evaluate

# ---------- モデル・プロセッサの初期化 ----------
gemma_model_id = "google/gemma-3-4b-it"

gemma_model = Gemma3ForConditionalGeneration.from_pretrained(
    gemma_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
).eval()

gemma_processor = AutoProcessor.from_pretrained(gemma_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

## ベンチマーク評価

## JA-VLM-Bench-In-the-Wild


In [ ]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
def generate_vqa_answer(image, question, gt_answer=None):

    try:
        prompt = (
            "<start_of_image><end_of_image>\n"
            f"この画像について「{question.strip()}」という質問に対して、簡潔に日本語で答えてください。"
        )
        inputs = gemma_processor(
            text=prompt,
            images=image,
            return_tensors="pt"
        )
        inputs = {k: v.to(gemma_model.device) for k, v in inputs.items()}

        with torch.inference_mode():
            outputs = gemma_model.generate(
                **inputs,
                max_new_tokens=100,
                do_sample=False
            )

        pred_answer = gemma_processor.decode(outputs[0], skip_special_tokens=True)
        pred_answer = pred_answer.strip().split("\n")[-1].strip()

    except Exception as e:
        print(f" Error during inference: {e}")
        pred_answer = "ERROR"

    return {
        "question": question,
        "gt_answer": gt_answer,
        "pred_answer": pred_answer.strip()
    }



In [ ]:
def infer_ja_vg_vqa500(dataset, image_field="image", qa_field="qas"):
    results = []

    for img_idx, item in enumerate(dataset):
        image = item[image_field].convert("RGB")

        print(f"\n====== [VQA] Image {img_idx} の質問処理 ======")

        for qa_idx, qa in enumerate(item[qa_field]):
            question = qa["question"]
            gt_answer = qa.get("answer", None)

            result = generate_vqa_answer(image, question, gt_answer)
            results.append(result)

            # 中間出力（統一形式）
            print(f"[Image {img_idx} | QA {qa_idx}] 質問: {question}")
            print(f"           正解      : {gt_answer}")
            print(f"           生成回答  : {result['pred_answer']}\n")

    return results

In [ ]:
# ---------- 評価処理（ROUGE） ----------
def evaluate_metric(results, bench_name):
    """
    推論結果を ROUGE-L で評価する。
    """
    rouge = evaluate.load("rouge")

    # 有効なペアのみ抽出
    valid_pairs = [
        (r["pred_answer"], r["gt_answer"])
        for r in results
        if r["gt_answer"] is not None and r["pred_answer"] != "ERROR"
    ]

    if not valid_pairs:
        print(f"[Warning] {bench_name} に評価可能な結果がありません。")
        return

    predictions, references = zip(*valid_pairs)

    print(f"\n Evaluation for {bench_name} (ROUGE-L)")

    score = rouge.compute(
        predictions=predictions,
        references=references,
        rouge_types=["rougeL"]
    )

    print(f" - Rouge-L Score : {score['rougeL']:.4f}")

In [ ]:
from datasets import load_dataset
# ========= 評価開始 ==========
# 1. JA-VG-VQA-500
vg_dataset = load_dataset("SakanaAI/JA-VG-VQA-500", split="test")

In [ ]:
vg_dataset

Dataset({
    features: ['image_id', 'url', 'width', 'height', 'coco_id', 'flickr_id', 'qas', 'image'],
    num_rows: 500
})

In [ ]:
vg_results = infer_ja_vg_vqa500(vg_dataset)


====== [VQA] Image 0 の質問処理 ======
[Image 0 | QA 0] 質問: 場所はどこになりますか？
           正解      : 車のある屋外の場所
           生成回答  : この画像は、オーストラリアの海岸沿いの道路に駐車された白いトラックの前に置かれたサーフボードを撮影したものです。背景には海と砂浜が見えます。

[Image 0 | QA 1] 質問: 時間帯はいつごろですか？
           正解      : 昼間
           生成回答  : 時間帯は不明

[Image 0 | QA 2] 質問: どんな天気をしていますか？
           正解      : 晴れている
           生成回答  : 晴れです。

[Image 0 | QA 3] 質問: 車は何台ありますか？
           正解      : 1台
           生成回答  : 1台です。

[Image 0 | QA 4] 質問: 車はどんな色をしていますか？
           正解      : 白色
           生成回答  : 白い車です。

[Image 0 | QA 5] 質問: サーフボードはいくつありますか？
           正解      : 1つ
           生成回答  : 画像にはサーフボードが1枚あります。

[Image 0 | QA 6] 質問: サーフボードはどんな色をしていますか？
           正解      : 白色
           生成回答  : サーフボードは白です。

[Image 0 | QA 7] 質問: 車の向こう側には何が見えますか？
           正解      : 青い海と青い空が広がっているのが見える
           生成回答  : 車の向こう側には、海と砂浜が見えます。


====== [VQA] Image 1 の質問処理 ======
[Image 1 | QA 0] 質問: ここはどこですか？
           正解      : 池のほとりです。
           生成回答  : この画像は、イギリスのロンドンにある、ボタニック・ガーデンに隣接する、

SakanaAI/JA-VG-VQA-500のデータすべて使って実験したところ、
上記の状況下でColab の使用量上限に達した。


In [ ]:
for r in vg_results[:10]:
    print(r)

{'question': '場所はどこになりますか？', 'gt_answer': '車のある屋外の場所', 'pred_answer': 'この画像は、オーストラリアの海岸沿いの道路に駐車された白いトラックの前に置かれたサーフボードを撮影したものです。背景には海と砂浜が見えます。'}
{'question': '時間帯はいつごろですか？', 'gt_answer': '昼間', 'pred_answer': '時間帯は不明'}
{'question': 'どんな天気をしていますか？', 'gt_answer': '晴れている', 'pred_answer': '晴れです。'}
{'question': '車は何台ありますか？', 'gt_answer': '1台', 'pred_answer': '1台です。'}
{'question': '車はどんな色をしていますか？', 'gt_answer': '白色', 'pred_answer': '白い車です。'}
{'question': 'サーフボードはいくつありますか？', 'gt_answer': '1つ', 'pred_answer': '画像にはサーフボードが1枚あります。'}
{'question': 'サーフボードはどんな色をしていますか？', 'gt_answer': '白色', 'pred_answer': 'サーフボードは白です。'}
{'question': '車の向こう側には何が見えますか？', 'gt_answer': '青い海と青い空が広がっているのが見える', 'pred_answer': '車の向こう側には、海と砂浜が見えます。'}
{'question': 'ここはどこですか？', 'gt_answer': '池のほとりです。', 'pred_answer': 'この画像は、イギリスのロンドンにある、ボタニック・ガーデンに隣接する、リバー・グレン（River Glen）の風景です。'}
{'question': '池には何がいますか？', 'gt_answer': '鴨がいます。', 'pred_answer': '池には、水面に浮かぶアヒルがいます。'}


In [ ]:
evaluate_metric(vg_results, "JA-VG-VQA-500")


 Evaluation for JA-VG-VQA-500 (ROUGE-L)
 - Rouge-L Score : 0.0833


上記の0.0833は</br>
vg_dataset = load_dataset("SakanaAI/JA-VG-VQA-500", split="test[:3]")</br>
つまり、データの先頭3件だけ取得したときの結果となっている

## JA-VLM-Bench-In-the-Wild

In [ ]:
# 2. JA-VLM-Bench-In-the-Wild
vlm_dataset = load_dataset("SakanaAI/JA-VLM-Bench-In-the-Wild", split="test")

In [ ]:
vlm_dataset

Dataset({
    features: ['page_url', 'image_url', 'image', 'question', 'answer'],
    num_rows: 3
})

In [ ]:
def infer_ja_vlm_bench_wild(dataset):
    """
    1画像1質問の形式（SakanaAI/JA-VLM-Bench-In-the-Wild）に対応した推論関数。
    各行に対して画像+質問を渡し、推論結果と正解を記録する。
    """
    results = []

    for idx, item in enumerate(dataset):
        image = item["image"].convert("RGB")
        question = item["question"]
        gt_answer = item.get("answer", None)

        print(f"\n====== [VQA] Entry {idx} - 質問処理開始 ======")

        result = generate_vqa_answer(image, question, gt_answer)
        results.append(result)

        # 中間出力（統一形式）
        print(f"[Entry {idx}] 質問: {question}")
        print(f"           正解      : {gt_answer}")
        print(f"           生成回答  : {result['pred_answer']}\n")

    return results

In [ ]:
vlm_results = infer_ja_vlm_bench_wild(vlm_dataset)


====== [VQA] Entry 0 - 質問処理開始 ======
[Entry 0] 質問: この写真はどこの国で撮影されましたか？
           正解      : この写真は日本で撮影されました。
           生成回答  : 日本


====== [VQA] Entry 1 - 質問処理開始 ======
[Entry 1] 質問: この自動販売機にはどのブランドの飲料が含まれていますか？
           正解      : キリンの飲料が含まれています。
           生成回答  : キリン


====== [VQA] Entry 2 - 質問処理開始 ======
[Entry 2] 質問: この写真に写っている有名な建物の名前は何ですか？
           正解      : 東京タワーです。
           生成回答  : 東京タワーです。


====== [VQA] Entry 3 - 質問処理開始 ======
[Entry 3] 質問: この写真にはどんな植物が映っていますか？
           正解      : 桜の花が映っています。
           生成回答  : 桜の花が映っています。


====== [VQA] Entry 4 - 質問処理開始 ======
[Entry 4] 質問: この写真に写っているタワーは何ですか？
           正解      : この写真には東京スカイツリーが写っています。
           生成回答  : 東京タワーです。


====== [VQA] Entry 5 - 質問処理開始 ======
[Entry 5] 質問: この駅は何線のものですか？
           正解      : 画像から正確な路線名は読み取れませんが、京都へ行く電車のある駅であることが分かります。
           生成回答  : この駅は4線の駅です。


====== [VQA] Entry 6 - 質問処理開始 ======
[Entry 6] 質問: この駅のホームには何番線がありますか？
           正解      : 1番線と2番線があります。
           生成回答  : 2番線です。


====== [

In [ ]:
for r in vlm_results[:10]:
    print(r)

{'question': 'この写真はどこの国で撮影されましたか？', 'gt_answer': 'この写真は日本で撮影されました。', 'pred_answer': '日本'}
{'question': 'この自動販売機にはどのブランドの飲料が含まれていますか？', 'gt_answer': 'キリンの飲料が含まれています。', 'pred_answer': 'キリン'}
{'question': 'この写真に写っている有名な建物の名前は何ですか？', 'gt_answer': '東京タワーです。', 'pred_answer': '東京タワーです。'}
{'question': 'この写真にはどんな植物が映っていますか？', 'gt_answer': '桜の花が映っています。', 'pred_answer': '桜の花が映っています。'}
{'question': 'この写真に写っているタワーは何ですか？', 'gt_answer': 'この写真には東京スカイツリーが写っています。', 'pred_answer': '東京タワーです。'}
{'question': 'この駅は何線のものですか？', 'gt_answer': '画像から正確な路線名は読み取れませんが、京都へ行く電車のある駅であることが分かります。', 'pred_answer': 'この駅は4線の駅です。'}
{'question': 'この駅のホームには何番線がありますか？', 'gt_answer': '1番線と2番線があります。', 'pred_answer': '2番線です。'}
{'question': '画像に何が見えますか？', 'gt_answer': '日本の伝統的な建物の前に野菜と果物が置かれてある光景が見えます。店の前には、レモンやトマト、ネギなどが置いてあり、提灯とメニューが掲げられています。', 'pred_answer': 'この画像には、日本の古い町並みに佇む食堂の入り口が見えます。食堂の正面には、木製の壁に格子戸があり、メニューが掲示されています。正面には、野菜や玉ねぎなどが積まれた木製のテーブルと椅子があり、食堂の食材をアピールしています。また、提灯や看板など、日本の伝統的な要素も確認できます。'}
{'question': 'この画像には何個の提灯がありますか？

In [ ]:
evaluate_metric(vlm_results, "JA-VLM-Bench-In-the-Wild")


 Evaluation for JA-VLM-Bench-In-the-Wild (ROUGE-L)
 - Rouge-L Score : 0.1844


## ファインチューニング

In [ ]:
from datasets import load_dataset

dataset = load_dataset("line-corporation/JIC-VQA", split="train[:10]")
print(dataset.features)

README.md:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

jafacility20.csv:   0%|          | 0.00/291k [00:00<?, ?B/s]

jaflower30.csv:   0%|          | 0.00/664k [00:00<?, ?B/s]

jafood101.csv:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

jalandmark10.csv:   0%|          | 0.00/324k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7654 [00:00<?, ? examples/s]

{'id': Value(dtype='int64', id=None), 'license': Value(dtype='string', id=None), 'license_url': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None), 'category': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None)}


In [ ]:
print(dataset[0])

{'id': 11190751074, 'license': 'Attribution License', 'license_url': 'https://creativecommons.org/licenses/by/2.0/', 'url': 'https://live.staticflickr.com/5536/11190751074_f97587084e_o.jpg', 'category': 'ガソリンスタンド', 'question': "この画像にはどの施設が映っていますか？次の四つの選択肢から正しいものを選んでください: ['スーパーマーケット', 'コンビニ', '駐車場', 'ガソリンスタンド']", 'messages': [{'content': 'あなたは優秀な日本語ビジュアルアシスタントです。画像についての質問に丁寧に答えてください。', 'role': 'system'}, {'content': "<image> この画像にはどの施設が映っていますか？次の四つの選択肢から正しいものを選んでください: ['スーパーマーケット', 'コンビニ', '駐車場', 'ガソリンスタンド']", 'role': 'user'}, {'content': 'ガソリンスタンド', 'role': 'assistant'}], 'text': "<s>[SYSTEM] あなたは優秀な日本語ビジュアルアシスタントです。画像についての質問に丁寧に答えてください。\n[USER] <image> この画像にはどの施設が映っていますか？次の四つの選択肢から正しいものを選んでください: ['スーパーマーケット', 'コンビニ', '駐車場', 'ガソリンスタンド']\n[ASSISTANT] ガソリンスタンド</s>"}


ファインチューニングをするための実装を実験していましたが、上手くいかなかったため断念

## 苦戦した点

本プロジェクトを進める中で、いくつかの技術的・運用的な課題に直面し、解決に時間を要した。以下に主な苦戦点を整理する。

1. 画像とテキストを組み合わせてモデルに入力し、推論を行う実装の構築に時間がかかった。
マルチモーダルな入力を正しく処理するためのプロンプト設計や、前処理（画像のRGB変換など）に習熟する必要があり、試行錯誤を繰り返した。

2. ベンチマーク評価の方法を調査するのに時間を要した。
特にROUGEスコアなどの自動評価指標の扱い方や、どのような条件で評価が行われるのかを理解するまでに苦労した。

3. ベンチマーク評価の結果に違和感があり、コードや処理ロジックの確認、さらにはGPUメモリの制限による処理失敗への対応に時間を要した。
データセットの件数が多くなるとGPUメモリを超過するため、バッチサイズや処理単位の調整を検討する必要があった。

4. Heron-Benchについて調査した際、GPT-4を用いた評価プロセス（生成された2つの回答を比較し、GPT-4が点数を付与する方式）の理解に時間がかかった。
さらに、GPT-4の使用にはコストがかかるため、予算の計算やAPI利用準備などを含めた検討も必要となった。結果的に、この評価方法は後回しとし、今回は実施を見送った。

5. VLM（Vision-Language Model）のファインチューニングを可能にする実装方法の調査にも多くの時間を要した。
特に、マルチモーダル入力に対応するモデル構造の改変や、LoRAなどの軽量ファインチューニング手法の適用に多くの技術的課題が存在し、実装が思うように進まなかった。





## 参考資料：

*   https://huggingface.co/google/gemma-3-4b-it
*   https://huggingface.co/datasets/SakanaAI/JA-VLM-Bench-In-the-Wild
*   https://huggingface.co/datasets/SakanaAI/JA-VG-VQA-500
*   https://huggingface.co/datasets/turing-motors/Japanese-Heron-Bench
*   https://huggingface.co/datasets/line-corporation/JIC-VQA
*   https://qiita.com/tinymouse/items/1c5d49579e95369b1d0b
*   https://note.com/owlet_notes/n/na7416b0414b7
*   https://zenn.dev/watamoo/articles/0590ad46d7b26b
